In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

#### Comparing Histograms (Recap)

In [ ]:
#Load data
goals = Table.read_table('big5.csv').select(3, 9, 6, 10)
#relabel columns
goals = goals.relabeled(1, 'Home').relabeled(3, 'Away')
goals

In [ ]:
# Recap: What does height represent/what does area represent?
goals.hist('Away', unit = 'goals')

In [ ]:
climate = Table.read_table('NOAA.csv').relabeled(5, '1948').relabeled(6, '2018')
#How to get actual counts on the y-axis instead of percent/unit
#Does it always make sense?
climate.hist(5, 6, unit = 'days above 90 degrees F', 
             bins = make_array(0,10,30,60,80,100)) 

## Groups

In [ ]:
# load cones.csv
all_cones = Table.read_table('cones.csv')
all_cones

In [ ]:
all_cones.column('Price')/2

In [ ]:
# TODO: recap creating/ applying a function
# create function to half the prices of columns above and add the result as a new column
def half(myArray):
    return np.round(myArray/2,2)


In [ ]:
newPrices = half(all_cones.column('Price'))
# newPrices = all_cones.apply(half, 'Price')

In [ ]:
all_cones.with_column('reduced Price', newPrices)

In [ ]:
# TODO: group by 'Flavor'
all_cones.select(0,2).group('Flavor',np.average)


In [ ]:
# TODO: group by 'Flavor', but now looking at average (Price) instead of count/len


## Grouping by 2 attributes/columns

In [ ]:
#TODO: group by flavor and color

In [ ]:
all_cones.group(make_array('Flavor', 'Color'), list)

### Let's look at 2 categorical variables in a bigger data set
Grouping by 2 different attributes/columns

In [ ]:
nba = Table.read_table('nba_salaries.csv').relabeled(4, 'SALARY')
nba

#### Question: Which NBA teams spent the most on their “starters” in 2015-2016?
Assum the “starter” for a team & position is the player with the highest salary on that team in that position


In [ ]:
#TODO: drop data that we are not interested in
nba.drop(0,1).group(['TEAM','POSITION'], max).sort('SALARY max', descending = True)

In [ ]:
#TODO: group by TEAM & position, should we just look at the count?


## Pivot: another way to look at 2 categorical variables

In [ ]:
#TODO: Revisit our simple table of cones
all_cones.pivot('Color','Flavor', 'Price', min)


In [ ]:
#TODO: Can we look at something else but count? Maybe the average price?


In [ ]:
nba

#### Revisit Question: Which NBA teams spent the most on their “starters” in 2015-2016?
Assum the “starter” for a team & position is the player with the highest salary on that team in that position

In [ ]:
nba.drop(0,1).pivot('POSITION', 'TEAM', 'SALARY', max)

### Comparing distributions, answering questions about associations
Let's use what we learned using group, pivot, functions
Data set from https://data.ca.gov/dataset/ca-educational-attainment-personal-income/resource/26201f19-4469-4311-a819-bbbd3e557eda
Represents the education and income data for California Adults (2008 - 2014)

In [ ]:
full_table = Table.read_table('educ_inc.csv')
full_table

In [ ]:
#Only look at adults (18+) in 2014
ca_2014 = full_table.where('Year', are.equal_to('1/1/14 0:00')).where(
    'Age', are.not_equal_to('00 to 17')).drop(0)
ca_2014

In [ ]:
#Let's focus in on adults qho are between 18 - 64, 
#let's narrow in on that age group
ca_2014 = ca_2014.where('Age', '18 to 64')
ca_2014
#TODO: can we look at just 2 attributes/columns and use pivot to cross-classify
# step 1: drop unnecessary 
edu_Income = ca_2014.drop(0,1)

#step 2: use pivot
pivot_eduIncome = edu_Income.pivot('Educational Attainment','Personal Income', 'Population Count', sum)
pivot_eduIncome


In [ ]:
# Is this information easy to interpret? How about looking at percent?

In [ ]:
def percent(values):
    return np.round(values/sum(values) * 100,2)

In [ ]:
#TODO: use the function percent to create a distribution table
distribution = pivot_eduIncome. select(0).with_columns(
    'BS or higher', percent(pivot_eduIncome.column(1)),
    '< 4 year degree', percent(pivot_eduIncome.column(2)),
    'High school', percent(pivot_eduIncome.column(3)))
distribution

In [ ]:
distribution.barh('Personal Income')

In [ ]:
#TODO: visualize to see if there is an association (barh())

## Combining 2 tables using join

In [ ]:
# Create a table of drinks
drinks = Table(['Drink','Cafe','Price']).with_rows([
    ['Milk Tea', 'Tea One', 4],
    ['Espresso', 'Nefeli', 2],
    ['Latte', 'Nefeli', 3],
    ['Espresso', "Abe's", 2]])
drinks

In [ ]:
discounts = Table(). with_columns(
    'Coupon % off', make_array(25,50,5),
    'Location', make_array('Tea One', 'Nefeli', 'Tea One'))
discounts

In [ ]:
#TODO: create a combined table using join
newDrinks = drinks.join('Cafe',discounts,'Location' )
newDrinks

In [ ]:
#TODO: add a reduced price column
reducedPrice = newDrinks.column('Price') - (newDrinks.column('Coupon % off') / 100) * newDrinks.column('Price')
newDrinks.with_column('Reduced', reducedPrice)

## SF Bike Share Example

In [ ]:
#Ride share example
trips = Table.read_table('trips_2.csv')
trips

In [ ]:
#TODO: try visualizing a histogram of duration of bike use, is this useful?
trips.hist('Duration')

In [ ]:
#TODO: let's narrow it down to useful data (maybe duration up to 30 min)
# minutes or less: 30*60 = 1800 seconds
# Try histogram again, what can we learn from this histogram?
commute = trips.where('Duration', are.below(1800))
commute.hist('Duration')

In [ ]:
# how about cross-classifying? let's look at most popular start station


In [ ]:
# How about most popular start to end?
